In [1]:
import os
import time
import importlib
import tqdm

import alpha_vantage
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

import catboost
from catboost import CatBoostClassifier
import pandas as pd

import common
importlib.reload(common)
from common import chech_if_should_buy, get_technical_indicators_df

# Prepare API

In [2]:
key = 'W79HC61KBLPCXW6C'

In [3]:
ti = TechIndicators(key, output_format='pandas')
ts = TimeSeries(key, output_format='pandas')

# Get S&P 500 companies' symbols

In [4]:
with open('../sp_500_symbols.txt', 'r') as f:
    sp_500_symbols = f.read().split('\n')[:-1]

In [5]:
print(len(sp_500_symbols))
print(sp_500_symbols[:5])

505
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


# Identify stocks to buy 

In [6]:
classifier = CatBoostClassifier()
models_path = 'models'
model_path = os.path.join(models_path, 'model_v3')
classifier.load_model(model_path)

In [7]:
tech_ind_df_offset = pd.read_csv(os.path.join(models_path, 'model_v3_offset'), header = None, 
                                 index_col = 0, squeeze = True).drop('target')
tech_ind_df_scale = pd.read_csv(os.path.join(models_path, 'model_v3_scale'), header = None, 
                                index_col = 0, squeeze = True).drop('target')

In [14]:
tech_ind_df_offset

0
adx_14         2.428294e+01
adx_7          3.362053e+01
aroon_down     4.499704e+01
aroon_up       5.367336e+01
cci_14         1.158711e+01
cci_7          7.983719e+00
chaikin_a/d    9.838181e+07
dema_13        6.863500e+01
dema_26        6.863493e+01
mfi_14         5.217659e+01
mfi_7          5.221155e+01
mom_15         2.196574e-01
mom_8          1.178061e-01
obv            8.236932e+07
rsi_14         5.201486e+01
rsi_7          5.225535e+01
slowd          5.468428e+01
slowk          5.468183e+01
sma_13         6.854891e+01
sma_26         6.845607e+01
sma_9          6.857816e+01
Name: 1, dtype: float64

In [15]:
tech_ind_df_scale

0
adx_14         9.698831e+00
adx_7          1.279796e+01
aroon_down     3.618896e+01
aroon_up       3.643883e+01
cci_14         1.087209e+02
cci_7          1.001543e+02
chaikin_a/d    8.296707e+08
dema_13        1.305306e+02
dema_26        1.305731e+02
mfi_14         1.608732e+01
mfi_7          2.206016e+01
mom_15         1.320058e+01
mom_8          9.824388e+00
obv            5.100615e+08
rsi_14         1.208676e+01
rsi_7          1.680107e+01
slowd          2.716099e+01
slowk          2.799890e+01
sma_13         1.302688e+02
sma_26         1.300528e+02
sma_9          1.303411e+02
Name: 1, dtype: float64

In [16]:
classifier.feature_names_

['adx_14',
 'adx_7',
 'aroon_down',
 'aroon_up',
 'cci_14',
 'cci_7',
 'chaikin_a/d',
 'dema_13',
 'dema_26',
 'mfi_14',
 'mfi_7',
 'mom_15',
 'mom_8',
 'obv',
 'rsi_14',
 'rsi_7',
 'slowd',
 'slowk',
 'sma_13',
 'sma_26',
 'sma_9']

In [47]:
def identify_stocks_to_buy_with_model(ti, symbols, model, tech_ind_df_offset, tech_ind_df_scale):
    should_process = True
    first_stock_to_process = None
#     first_stock_to_process = 'CBS'
    
    stocks_to_buy_ti = []
    stocks_to_buy = []
    predictions = []
    for stock in tqdm.tqdm(symbols):
        start = time.time()
        print('Stock: {}'.format(stock))
        
        if not should_process or stock == 'BF.B':
            if stock == first_stock_to_process:
                should_process = True
            print('Cannot process')
            continue
            
        tech_ind = get_technical_indicators_df(ti, stock).drop(['mom_1'], axis=1)
        data_x = (tech_ind.iloc[-1:] - tech_ind_df_offset) / tech_ind_df_scale
        print(data_x)
        
        pred = classifier.predict_proba(data_x)[0][1]
        print('Prediction: {}'.format(pred))
        
        should_buy = True if pred > 0.9 else False
        if should_buy:
            stocks_to_buy_ti.append(data_x)
            stocks_to_buy.append(stock)
            predictions.append(pred)
            
        end = time.time()
        process_time = end - start
        print('Time: {}'.format(process_time))
        time.sleep(max(60 - process_time, 1))  # API calls frequency restrictions
        
    return stocks_to_buy_ti, stocks_to_buy, predictions

In [48]:
stocks_to_buy_ti, stocks_to_buy, predictions = identify_stocks_to_buy_with_model(
    ti, 
    sp_500_symbols, 
    classifier, 
    tech_ind_df_offset, 
    tech_ind_df_scale
)











  0%|          | 0/505 [00:00<?, ?it/s]

Stock: MMM


KeyboardInterrupt: 

In [33]:
stock_pred = sorted(zip(stocks_to_buy, predictions), key=lambda x: x[1], reverse=True)
for s, p in stock_pred:
    print('Stock: {} | Pred: {}'.format(s, p))

Stock: b | Pred: 4
Stock: c | Pred: 3
Stock: a | Pred: 1


### Intuition

In [33]:
def identify_stocks_to_buy(ti, symbols):
    should_process = False
    first_stock_to_process = 'AVGO'
    
    stocks_to_buy_ti = []
    stocks_to_buy = []
    for stock in tqdm.tqdm(symbols):
        if not should_process:
            if stock == first_stock_to_process:
                should_process = True
            continue
        
        stock_ti_df, should_buy = chech_if_should_buy(ti, stock)
        if should_buy:
            stocks_to_buy_ti.append(stock_ti_df)
            stocks_to_buy.append(stock)
            print(stock)
#         time.sleep(30)  # API calls frequency restrictions
        
    return stocks_to_buy_ti, stocks_to_buy

In [34]:
stocks_to_buy_ti, stocks_to_buy = identify_stocks_to_buy(ti, sp_500_symbols)



  0%|          | 0/505 [00:00<?, ?it/s]

 17%|█▋        | 84/505 [00:00<00:03, 121.12it/s]

Stock: BF.B - error
Stock: CHRW
              macd  macd_hist  macd_signal      rsi   chaikin_a/d
date                                                             
2020-12-07 -0.5993     0.2858      -0.8851  48.3171  3.069651e+08
2020-12-08 -0.4540     0.3449      -0.7989  52.0560  3.074221e+08
Stock: CA
            macd_hist    macd  macd_signal      rsi   chaikin_a/d
date                                                             
2018-11-05     0.0623  0.1497       0.0874  63.3076  5.771658e+08
2018-11-06     0.0574  0.1591       0.1017  63.3076  5.771658e+08
Stock: COG
              macd  macd_hist  macd_signal      rsi   chaikin_a/d
date                                                             
2020-12-07 -0.2967    -0.0667      -0.2300  42.2530  1.508906e+08
2020-12-08 -0.3191    -0.0713      -0.2478  41.3278  1.510525e+08




 17%|█▋        | 88/505 [00:18<09:24,  1.35s/it] 

Stock: CDNS
            macd_hist    macd  macd_signal      rsi   chaikin_a/d
date                                                             
2020-12-07     0.3942  1.2239       0.8297  59.7701  8.999582e+08
2020-12-08     0.3947  1.3231       0.9283  58.1552  9.000313e+08




 18%|█▊        | 89/505 [00:23<15:41,  2.26s/it]

...

Stock: ZBH
            macd_signal  macd_hist    macd      rsi   chaikin_a/d
date                                                             
2020-12-08       1.5602    -0.5293  1.0309  50.6287  3.013875e+08
2020-12-09       1.4368    -0.4934  0.9435  51.7551  3.020239e+08




100%|█████████▉| 504/505 [34:38<00:06,  6.97s/it]

Stock: ZION
              macd  macd_hist  macd_signal      rsi   chaikin_a/d
date                                                             
2020-12-08  2.3331     0.1460       2.1871  64.1943  1.616833e+08
2020-12-09  2.2835     0.0771       2.2064  63.1829  1.604453e+08




100%|██████████| 505/505 [34:45<00:00,  6.98s/it]

Stock: ZTS
            macd_signal    macd  macd_hist      rsi   chaikin_a/d
date                                                             
2020-12-08      -0.8332 -1.4464    -0.6133  43.4434  2.355375e+08
2020-12-09      -0.9592 -1.4635    -0.5043  42.8519  2.353030e+08
